In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fuzzywuzzy

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 320.4MB 52kB/s 


In [ ]:
!pip install sister
import sister

     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
  Created wheel for sister: filename=sister-0.1.10-cp36-none-any.whl size=5393 sha256=e31833b50c11318c60523ee81b43780ee5529043e782463609aa9265d393a89a
  Stored in directory: /root/.cache/pip/wheels/69/8d/01/c066cc2e5c98891fc1af11f393882ee718b6026f3f8b0623b0
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3044245 sha256=0608a5ba5b8c863cb9b3a8b6a9f8069709351f5b184fab4ed0f025da37fd714c
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built sister fasttext


In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.svm import SVR
# from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import warnings; warnings.filterwarnings("ignore");
import time
from sklearn.model_selection import GridSearchCV
from sklearn import pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer
#from nltk.metrics import edit_distance
from nltk.stem.porter import *
stemmer = PorterStemmer()
import re
from sklearn.metrics.pairwise import cosine_similarity

import random
random.seed(47)

### Loading Data

In [ ]:
train_data = pd.read_csv(r'/content/drive/My Drive/project/train.csv', encoding="ISO-8859-1")
test_data = pd.read_csv(r'/content/drive/My Drive/project/test.csv', encoding="ISO-8859-1")
product_desc = pd.read_csv(r'/content/drive/My Drive/project/product_descriptions.csv', encoding="ISO-8859-1")
attributes = pd.read_csv(r'/content/drive/My Drive/project/attributes.csv', encoding="ISO-8859-1")

brand = attributes[attributes.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
num_train = train_data.shape[0]

all_data = pd.concat((train_data, test_data), axis=0, ignore_index=True)
all_data = pd.merge(all_data, product_desc, how='left', on='product_uid')
all_data = pd.merge(all_data, brand, how='left', on='product_uid')

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
use_model = hub.load(module_url)


In [ ]:
!mkdir GloVe
!curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip GloVe/glove.840B.300d.zip -d GloVe/
!mkdir fastText
!curl -Lo fastText/crawl-300d-2M.vec.zip https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
!unzip fastText/crawl-300d-2M.vec.zip -d fastText/
!mkdir encoder
!curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
!curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl
import torch
import nltk
nltk.download('punkt')
from models import InferSent
V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 128, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))
infersent = infersent.cuda()
W2V_PATH = 'fastText/crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)
infersent.build_vocab_k_words(K=100000)

In [ ]:
# import time

In [ ]:
# start = time.time()
# with tf.device('/device:GPU:0'):
#   embeddings = use_model(all_data['product_description'].values[:10000])
# print(time.time()-start)

In [ ]:
all_data.shape

(240760, 7)

In [ ]:
all_data.head()

,id,product_uid,product_title,search_term,relevance,product_description,brand
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...,Delta
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...,Delta


### Feature Extraction

In [ ]:
def jaccard_similarity(query, document):
    intersectionn = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersectionn)/len(union)
def intersection(query, document):
    return len(set(query).intersection(set(document)))
    
embedder = sister.MeanEmbedding(lang="en")
def fast_text_similarity(str1,str2):
  return cosine_similarity([embedder(str1),embedder(str2)])[0][1]

def use_text_similarity(str1,str2):
  with tf.device('/device:GPU:0'):
    return cosine_similarity([use_model([str1])[0].numpy(),use_model([str2])[0].numpy()])[0][1]

def infer_text_similarity(str1,str2):
  return cosine_similarity([infersent.encode([str1])[0],infersent.encode([str2])[0]])[0][1]

Loading model...


In [ ]:
from fuzzywuzzy import fuzz
def fuzzy_partial_ratio(string_1 , string_2):
    return fuzz.partial_ratio(string_1, string_2)
def fuzzy_token_sort_ratio(string_1,string_2):
    return fuzz.token_sort_ratio(string_1,string_2)

In [ ]:
import json
with open(r'/content/drive/MyDrive/project/New document 2 (1).json','r') as f:
  temp = f.read()
temp = eval(temp)

In [ ]:
all_data['search_term'] = all_data['search_term'].map(lambda x:temp[x] if x in temp else x)

In [ ]:
stop_words = ['for', 'xbi', 'and', 'in', 'th','on','sku','with','what','from','that','less','er','ing'] #'electr','paint','pipe','light','kitchen','wood','outdoor','door','bathroom'
strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}

def str_stem(s): 
    if isinstance(s, str):
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
        s = s.lower()
        s = s.replace("  "," ")
        s = s.replace(",","") #could be number / segment later
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = s.replace(" x "," xbi ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = s.replace("  "," ")
        s = s.replace(" . "," ")
        #s = (" ").join([z for z in s.split(" ") if z not in stop_w])
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        
        s = s.lower()
        s = s.replace("toliet","toilet")
        s = s.replace("airconditioner","air conditioner")
        s = s.replace("vinal","vinyl")
        s = s.replace("vynal","vinyl")
        s = s.replace("skill","skil")
        s = s.replace("snowbl","snow bl")
        s = s.replace("plexigla","plexi gla")
        s = s.replace("rustoleum","rust-oleum")
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool ga")
        s = s.replace("whirlpoolstainless","whirlpool stainless")
        return s
    else:
        return "null"

def segment_words(s1, s2):
    s2 = s2.lower()
    s2 = re.sub("[^a-z0-9./]"," ", s2)
    s2 = [z for z in set(s2.split()) if len(z)>2]
    words = s1.lower().split(" ")
    s = []
    for word in words:
        if len(word)>3:
            s11 = []
            s11 += segment_it(word,s2,True)
            if len(s)>1:
                s += [z for z in s11 if z not in ['er','ing','s','less'] and len(z)>1]
            else:
                s.append(word)
        else:
            s.append(word)
    return (" ".join(s))

def segment_it(s1, txt_arr, t):
    st = s1
    r = []
    for j in range(len(s1)):
        for word in txt_arr:
            if word == s1[:-j]:
                r.append(s1[:-j])
                s1=s1[len(s1)-j:]
                r += segment_it(s1, txt_arr, False)
    if t:
        i = len(("").join(r))
        if not i==len(st):
            r.append(st[i:])
    return r

def common_word(s1, s2):
    words, cnt = s1.split(), 0
    for word in words:
        if s2.find(word)>=0:
            cnt+=1
    return cnt

def whole_word(s1, s2, i_):
    cnt = 0
    while i_ < len(s2):
        i_ = s2.find(s1, i_)
        if i_ == -1:
            return cnt
        else:
            cnt += 1
            i_ += len(s1)
    return cnt

In [ ]:
all_data['search_term'] = all_data['search_term'].map(lambda x:str_stem(x))
all_data['product_title'] = all_data['product_title'].map(lambda x:str_stem(x))
all_data['product_description'] = all_data['product_description'].map(lambda x:str_stem(x))
all_data['brand'] = all_data['brand'].map(lambda x:str_stem(x))
all_data.head()

,id,product_uid,product_title,search_term,relevance,product_description,brand
0,2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,not onli do angl make joint stronger they also...,simpson strong tie
1,3,100001,simpson strong tie 12 gaug angl,l bracket,2.50,not onli do angl make joint stronger they also...,simpson strong tie
2,9,100002,behr premium textur deckov 1gal. #sc 141 tugbo...,deck over,3.00,behr premium textur deckov is an innov solid c...,behr premium textur deckov
3,16,100005,delta vero 1 handl shower onli faucet trim kit...,rain shower head,2.33,updat your bathroom with the delta vero singl ...,delta
4,17,100005,delta vero 1 handl shower onli faucet trim kit...,shower onli faucet,2.67,updat your bathroom with the delta vero singl ...,delta


In [ ]:
all_data['product_info'] = all_data['search_term']+"\t"+all_data['product_title'] +"\t"+all_data['product_description']
all_data.head()

,id,product_uid,product_title,search_term,relevance,product_description,brand,product_info
0,2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,not onli do angl make joint stronger they also...,simpson strong tie,angl bracket\tsimpson strong tie 12 gaug angl\...
1,3,100001,simpson strong tie 12 gaug angl,l bracket,2.50,not onli do angl make joint stronger they also...,simpson strong tie,l bracket\tsimpson strong tie 12 gaug angl\tno...
2,9,100002,behr premium textur deckov 1gal. #sc 141 tugbo...,deck over,3.00,behr premium textur deckov is an innov solid c...,behr premium textur deckov,deck over\tbehr premium textur deckov 1gal. #s...
3,16,100005,delta vero 1 handl shower onli faucet trim kit...,rain shower head,2.33,updat your bathroom with the delta vero singl ...,delta,rain shower head\tdelta vero 1 handl shower on...
4,17,100005,delta vero 1 handl shower onli faucet trim kit...,shower onli faucet,2.67,updat your bathroom with the delta vero singl ...,delta,shower onli faucet\tdelta vero 1 handl shower ...


In [ ]:
all_data['st_title_jaccard'] = all_data['product_info'].map(lambda x:jaccard_similarity(x.split('\t')[0].split(),x.split('\t')[1].split()))
all_data['st_des_jaccard'] = all_data['product_info'].map(lambda x:jaccard_similarity(x.split('\t')[0].split(),x.split('\t')[2].split()))

all_data['st_title_inter'] = all_data['product_info'].map(lambda x:intersection(x.split('\t')[0].split(),x.split('\t')[1].split()))
all_data['st_des_inter'] = all_data['product_info'].map(lambda x:intersection(x.split('\t')[0].split(),x.split('\t')[2].split()))

all_data['st_title_fast'] = all_data['product_info'].map(lambda x:fast_text_similarity(x.split('\t')[0],x.split('\t')[1]))
all_data['st_des_fast'] = all_data['product_info'].map(lambda x:fast_text_similarity(x.split('\t')[0],x.split('\t')[2]))

all_data['st_title_use'] = all_data['product_info'].map(lambda x:use_text_similarity(x.split('\t')[0],x.split('\t')[1]))
all_data['st_des_use'] = all_data['product_info'].map(lambda x:use_text_similarity(x.split('\t')[0],x.split('\t')[2]))

all_data['st_title_infer'] = all_data['product_info'].map(lambda x:infer_text_similarity(x.split('\t')[0],x.split('\t')[1]))
all_data['st_des_infer'] = all_data['product_info'].map(lambda x:infer_text_similarity(x.split('\t')[0],x.split('\t')[2]))
all_data.head()

,id,product_uid,product_title,search_term,relevance,product_description,brand,product_info,st_title_jaccard,st_des_jaccard,st_title_inter,st_des_inter,st_title_fast,st_des_fast,st_title_use,st_des_use
0,2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,not onli do angl make joint stronger they also...,simpson strong tie,angl bracket\tsimpson strong tie 12 gaug angl\...,0.142857,0.010101,1,1,0.574275,0.555772,0.211663,0.104759
1,3,100001,simpson strong tie 12 gaug angl,l bracket,2.50,not onli do angl make joint stronger they also...,simpson strong tie,l bracket\tsimpson strong tie 12 gaug angl\tno...,0.000000,0.000000,0,0,0.350070,0.459846,0.082214,0.088094
2,9,100002,behr premium textur deckov 1gal. #sc 141 tugbo...,deck over,3.00,behr premium textur deckov is an innov solid c...,behr premium textur deckov,deck over\tbehr premium textur deckov 1gal. #s...,0.000000,0.008621,0,1,0.553073,0.560206,0.043383,0.210235
3,16,100005,delta vero 1 handl shower onli faucet trim kit...,rain shower head,2.33,updat your bathroom with the delta vero singl ...,delta,rain shower head\tdelta vero 1 handl shower on...,0.062500,0.011765,1,1,0.555042,0.581922,0.231551,0.193347
4,17,100005,delta vero 1 handl shower onli faucet trim kit...,shower onli faucet,2.67,updat your bathroom with the delta vero singl ...,delta,shower onli faucet\tdelta vero 1 handl shower ...,0.214286,0.036145,3,3,0.722477,0.644635,0.409337,0.316044


In [ ]:
all_data['len_of_query'] = all_data['search_term'].map(lambda x:len(x.split())).astype(np.int64)
all_data['len_of_title'] = all_data['product_title'].map(lambda x:len(x.split())).astype(np.int64)
all_data['len_of_description'] = all_data['product_description'].map(lambda x:len(x.split())).astype(np.int64)
all_data['len_of_brand'] = all_data['brand'].map(lambda x:len(x.split())).astype(np.int64)
all_data['search_term'] = all_data['product_info'].map(lambda x:segment_words(x.split('\t')[0],x.split('\t')[1]))
all_data.head()

,id,product_uid,product_title,search_term,relevance,product_description,brand,product_info,st_title_jaccard,st_des_jaccard,st_title_inter,st_des_inter,st_title_fast,st_des_fast,st_title_use,st_des_use,len_of_query,len_of_title,len_of_description,len_of_brand
0,2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,not onli do angl make joint stronger they also...,simpson strong tie,angl bracket\tsimpson strong tie 12 gaug angl\...,0.142857,0.010101,1,1,0.574275,0.555772,0.211663,0.104759,2,6,135,3
1,3,100001,simpson strong tie 12 gaug angl,l bracket,2.50,not onli do angl make joint stronger they also...,simpson strong tie,l bracket\tsimpson strong tie 12 gaug angl\tno...,0.000000,0.000000,0,0,0.350070,0.459846,0.082214,0.088094,2,6,135,3
2,9,100002,behr premium textur deckov 1gal. #sc 141 tugbo...,deck over,3.00,behr premium textur deckov is an innov solid c...,behr premium textur deckov,deck over\tbehr premium textur deckov 1gal. #s...,0.000000,0.008621,0,1,0.553073,0.560206,0.043383,0.210235,2,12,169,4
3,16,100005,delta vero 1 handl shower onli faucet trim kit...,rain shower head,2.33,updat your bathroom with the delta vero singl ...,delta,rain shower head\tdelta vero 1 handl shower on...,0.062500,0.011765,1,1,0.555042,0.581922,0.231551,0.193347,3,14,109,1
4,17,100005,delta vero 1 handl shower onli faucet trim kit...,shower onli faucet,2.67,updat your bathroom with the delta vero singl ...,delta,shower onli faucet\tdelta vero 1 handl shower ...,0.214286,0.036145,3,3,0.722477,0.644635,0.409337,0.316044,3,14,109,1


In [ ]:
all_data['query_in_title'] = all_data['product_info'].map(lambda x:whole_word(x.split('\t')[0],x.split('\t')[1],0))
all_data['query_in_description'] = all_data['product_info'].map(lambda x:whole_word(x.split('\t')[0],x.split('\t')[2],0))
all_data['query_last_word_in_title'] = all_data['product_info'].map(lambda x:common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[1]))
all_data['query_last_word_in_description'] = all_data['product_info'].map(lambda x:common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[2]))
all_data['word_in_title'] = all_data['product_info'].map(lambda x:common_word(x.split('\t')[0],x.split('\t')[1]))
all_data['word_in_description'] = all_data['product_info'].map(lambda x:common_word(x.split('\t')[0],x.split('\t')[2]))
all_data['attr'] = all_data['search_term']+"\t"+all_data['brand']
all_data['word_in_brand'] = all_data['attr'].map(lambda x:common_word(x.split('\t')[0],x.split('\t')[1]))

all_data['ratio_title'] = all_data['word_in_title']/all_data['len_of_query']
all_data['ratio_description'] = all_data['word_in_description']/all_data['len_of_query']
all_data['ratio_brand'] = all_data['word_in_brand']/all_data['len_of_brand']


In [ ]:
# all_data.to_csv(r'/content/drive/My Drive/project/all_data.csv')
all_data = pd.read_csv(r'/content/drive/My Drive/project/all_data.csv',index_col=[0])

In [ ]:
all_data.head()

,id,product_uid,product_title,search_term,relevance,product_description,brand,product_info,st_title_jaccard,st_des_jaccard,st_title_inter,st_des_inter,st_title_fast,st_des_fast,st_title_use,st_des_use,len_of_query,len_of_title,len_of_description,len_of_brand,query_in_title,query_in_description,query_last_word_in_title,query_last_word_in_description,word_in_title,word_in_description,attr,word_in_brand,ratio_title,ratio_description,ratio_brand
0,2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,not onli do angl make joint stronger they also...,simpson strong tie,angl bracket\tsimpson strong tie 12 gaug angl\...,0.142857,0.010101,1,1,0.574275,0.555772,0.211663,0.104759,2,6,135,3,0,0,0,0,1,1,angl bracket\tsimpson strong tie,0,0.500000,0.500000,0.00
1,3,100001,simpson strong tie 12 gaug angl,l bracket,2.50,not onli do angl make joint stronger they also...,simpson strong tie,l bracket\tsimpson strong tie 12 gaug angl\tno...,0.000000,0.000000,0,0,0.350070,0.459846,0.082214,0.088094,2,6,135,3,0,0,0,0,1,1,l bracket\tsimpson strong tie,0,0.500000,0.500000,0.00
2,9,100002,behr premium textur deckov 1gal. #sc 141 tugbo...,deck over,3.00,behr premium textur deckov is an innov solid c...,behr premium textur deckov,deck over\tbehr premium textur deckov 1gal. #s...,0.000000,0.008621,0,1,0.553073,0.560206,0.043383,0.210235,2,12,169,4,0,0,0,0,1,1,deck over\tbehr premium textur deckov,1,0.500000,0.500000,0.25
3,16,100005,delta vero 1 handl shower onli faucet trim kit...,rain shower head,2.33,updat your bathroom with the delta vero singl ...,delta,rain shower head\tdelta vero 1 handl shower on...,0.062500,0.011765,1,1,0.555042,0.581922,0.231551,0.193347,3,14,109,1,0,0,0,0,1,1,rain shower head\tdelta,0,0.333333,0.333333,0.00
4,17,100005,delta vero 1 handl shower onli faucet trim kit...,shower onli faucet,2.67,updat your bathroom with the delta vero singl ...,delta,shower onli faucet\tdelta vero 1 handl shower ...,0.214286,0.036145,3,3,0.722477,0.644635,0.409337,0.316044,3,14,109,1,1,0,1,1,3,3,shower onli faucet\tdelta,0,1.000000,1.000000,0.00


In [ ]:
all_data['fuzzy_ratio_in_title'] = all_data['product_info'].map(lambda x:fuzzy_partial_ratio(x.split('\t')[0],x.split('\t')[1]))
all_data['fuzzy_ratio_in_description'] = all_data['product_info'].map(lambda x:fuzzy_partial_ratio(x.split('\t')[0],x.split('\t')[2]))
all_data['fuzzy_token_sort_ratio_in_title'] = all_data['product_info'].map(lambda x:fuzzy_token_sort_ratio(x.split('\t')[0],x.split('\t')[1]))
all_data['fuzzy_token_sort_ratio_in_description'] = all_data['product_info'].map(lambda x:fuzzy_token_sort_ratio(x.split('\t')[0],x.split('\t')[2]))

In [ ]:
all_data.head()

,id,product_uid,product_title,search_term,relevance,product_description,brand,product_info,st_title_jaccard,st_des_jaccard,st_title_inter,st_des_inter,st_title_fast,st_des_fast,st_title_use,st_des_use,len_of_query,len_of_title,len_of_description,len_of_brand,query_in_title,query_in_description,query_last_word_in_title,query_last_word_in_description,word_in_title,word_in_description,attr,word_in_brand,ratio_title,ratio_description,ratio_brand,fuzzy_ratio_in_title,fuzzy_ratio_in_description,fuzzy_token_sort_ratio_in_title,fuzzy_token_sort_ratio_in_description
0,2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,not onli do angl make joint stronger they also...,simpson strong tie,angl bracket\tsimpson strong tie 12 gaug angl\...,0.142857,0.010101,1,1,0.574275,0.555772,0.211663,0.104759,2,6,135,3,0,0,0,0,1,1,angl bracket\tsimpson strong tie,0,0.500000,0.500000,0.00,50,33,33,1
1,3,100001,simpson strong tie 12 gaug angl,l bracket,2.50,not onli do angl make joint stronger they also...,simpson strong tie,l bracket\tsimpson strong tie 12 gaug angl\tno...,0.000000,0.000000,0,0,0.350070,0.459846,0.082214,0.088094,2,6,135,3,0,0,0,0,1,1,l bracket\tsimpson strong tie,0,0.500000,0.500000,0.00,20,33,10,1
2,9,100002,behr premium textur deckov 1gal. #sc 141 tugbo...,deck over,3.00,behr premium textur deckov is an innov solid c...,behr premium textur deckov,deck over\tbehr premium textur deckov 1gal. #s...,0.000000,0.008621,0,1,0.553073,0.560206,0.043383,0.210235,2,12,169,4,0,0,0,0,1,1,deck over\tbehr premium textur deckov,1,0.500000,0.500000,0.25,67,67,21,1
3,16,100005,delta vero 1 handl shower onli faucet trim kit...,rain shower head,2.33,updat your bathroom with the delta vero singl ...,delta,rain shower head\tdelta vero 1 handl shower on...,0.062500,0.011765,1,1,0.555042,0.581922,0.231551,0.193347,3,14,109,1,0,0,0,0,1,1,rain shower head\tdelta,0,0.333333,0.333333,0.00,62,31,31,0
4,17,100005,delta vero 1 handl shower onli faucet trim kit...,shower onli faucet,2.67,updat your bathroom with the delta vero singl ...,delta,shower onli faucet\tdelta vero 1 handl shower ...,0.214286,0.036145,3,3,0.722477,0.644635,0.409337,0.316044,3,14,109,1,1,0,1,1,3,3,shower onli faucet\tdelta,0,1.000000,1.000000,0.00,100,39,39,0


In [ ]:
all_data['brand'] = all_data['brand'].apply(lambda x: np.str_(x))
df_train = all_data.iloc[:num_train]
df_test = all_data.iloc[num_train:]

id_test = df_test['id']

y_train = df_train['relevance'].values

X_train =df_train[:]
X_test = df_test[:]

In [ ]:
X_train.head()

,id,product_uid,product_title,search_term,relevance,product_description,brand,product_info,st_title_jaccard,st_des_jaccard,st_title_inter,st_des_inter,st_title_fast,st_des_fast,st_title_use,st_des_use,len_of_query,len_of_title,len_of_description,len_of_brand,query_in_title,query_in_description,query_last_word_in_title,query_last_word_in_description,word_in_title,word_in_description,attr,word_in_brand,ratio_title,ratio_description,ratio_brand,fuzzy_ratio_in_title,fuzzy_ratio_in_description,fuzzy_token_sort_ratio_in_title,fuzzy_token_sort_ratio_in_description
0,2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,not onli do angl make joint stronger they also...,simpson strong tie,angl bracket\tsimpson strong tie 12 gaug angl\...,0.142857,0.010101,1,1,0.574275,0.555772,0.211663,0.104759,2,6,135,3,0,0,0,0,1,1,angl bracket\tsimpson strong tie,0,0.500000,0.500000,0.00,50,33,33,1
1,3,100001,simpson strong tie 12 gaug angl,l bracket,2.50,not onli do angl make joint stronger they also...,simpson strong tie,l bracket\tsimpson strong tie 12 gaug angl\tno...,0.000000,0.000000,0,0,0.350070,0.459846,0.082214,0.088094,2,6,135,3,0,0,0,0,1,1,l bracket\tsimpson strong tie,0,0.500000,0.500000,0.00,20,33,10,1
2,9,100002,behr premium textur deckov 1gal. #sc 141 tugbo...,deck over,3.00,behr premium textur deckov is an innov solid c...,behr premium textur deckov,deck over\tbehr premium textur deckov 1gal. #s...,0.000000,0.008621,0,1,0.553073,0.560206,0.043383,0.210235,2,12,169,4,0,0,0,0,1,1,deck over\tbehr premium textur deckov,1,0.500000,0.500000,0.25,67,67,21,1
3,16,100005,delta vero 1 handl shower onli faucet trim kit...,rain shower head,2.33,updat your bathroom with the delta vero singl ...,delta,rain shower head\tdelta vero 1 handl shower on...,0.062500,0.011765,1,1,0.555042,0.581922,0.231551,0.193347,3,14,109,1,0,0,0,0,1,1,rain shower head\tdelta,0,0.333333,0.333333,0.00,62,31,31,0
4,17,100005,delta vero 1 handl shower onli faucet trim kit...,shower onli faucet,2.67,updat your bathroom with the delta vero singl ...,delta,shower onli faucet\tdelta vero 1 handl shower ...,0.214286,0.036145,3,3,0.722477,0.644635,0.409337,0.316044,3,14,109,1,1,0,1,1,3,3,shower onli faucet\tdelta,0,1.000000,1.000000,0.00,100,39,39,0


In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words='english')
tsvd = TruncatedSVD(n_components=20, random_state = 2016)
st_tfidf = tsvd.fit_transform(tfidf.fit_transform(X_train['search_term']))
title_tfidf = tsvd.fit_transform(tfidf.fit_transform(X_train['product_title']))
brand_tfidf = tsvd.fit_transform(tfidf.fit_transform(X_train['brand']))
tsvd1 = TruncatedSVD(n_components=50, random_state = 2016)
prod_des_tfidf = tsvd1.fit_transform(tfidf.fit_transform(X_train['product_description']))

to_train = pd.DataFrame(X_train)
to_train = to_train.drop(['id','relevance','product_title','search_term','brand','product_description','product_info','attr'],axis =1)

In [ ]:
to_train.head()


,product_uid,st_title_jaccard,st_des_jaccard,st_title_inter,st_des_inter,st_title_fast,st_des_fast,st_title_use,st_des_use,len_of_query,len_of_title,len_of_description,len_of_brand,query_in_title,query_in_description,query_last_word_in_title,query_last_word_in_description,word_in_title,word_in_description,word_in_brand,ratio_title,ratio_description,ratio_brand,fuzzy_ratio_in_title,fuzzy_ratio_in_description,fuzzy_token_sort_ratio_in_title,fuzzy_token_sort_ratio_in_description
0,100001,0.142857,0.010101,1,1,0.574275,0.555772,0.211663,0.104759,2,6,135,3,0,0,0,0,1,1,0,0.500000,0.500000,0.00,50,33,33,1
1,100001,0.000000,0.000000,0,0,0.350070,0.459846,0.082214,0.088094,2,6,135,3,0,0,0,0,1,1,0,0.500000,0.500000,0.00,20,33,10,1
2,100002,0.000000,0.008621,0,1,0.553073,0.560206,0.043383,0.210235,2,12,169,4,0,0,0,0,1,1,1,0.500000,0.500000,0.25,67,67,21,1
3,100005,0.062500,0.011765,1,1,0.555042,0.581922,0.231551,0.193347,3,14,109,1,0,0,0,0,1,1,0,0.333333,0.333333,0.00,62,31,31,0
4,100005,0.214286,0.036145,3,3,0.722477,0.644635,0.409337,0.316044,3,14,109,1,1,0,1,1,3,3,0,1.000000,1.000000,0.00,100,39,39,0


In [ ]:
all_to_train = np.append(to_train.values,st_tfidf,axis=1)
all_to_train = np.append(all_to_train,title_tfidf,axis=1)
all_to_train = np.append(all_to_train,brand_tfidf,axis=1)
all_to_train = np.append(all_to_train,prod_des_tfidf,axis=1)


In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words='english')
tsvd = TruncatedSVD(n_components=20, random_state = 2016)
st_tfidf = tsvd.fit_transform(tfidf.fit_transform(X_test['search_term']))
title_tfidf = tsvd.fit_transform(tfidf.fit_transform(X_test['product_title']))
brand_tfidf = tsvd.fit_transform(tfidf.fit_transform(X_test['brand']))
tsvd1 = TruncatedSVD(n_components=50, random_state = 2016)
prod_des_tfidf = tsvd1.fit_transform(tfidf.fit_transform(X_test['product_description']))

to_test = pd.DataFrame(X_test)
to_test = to_test.drop(['id','relevance','product_title','search_term','brand','product_description','product_info','attr'],axis =1)

In [ ]:
all_to_test = np.append(to_test.values,st_tfidf,axis=1)
all_to_test = np.append(all_to_test,title_tfidf,axis=1)
all_to_test = np.append(all_to_test,brand_tfidf,axis=1)
all_to_test = np.append(all_to_test,prod_des_tfidf,axis=1)


In [ ]:
train_x, val_x, train_y, val_y = train_test_split(all_to_train, y_train, test_size=0.2, random_state=42)

### Models

In [ ]:
# from sklearn.ensemble import StackingRegressor
# from sklearn.linear_model import RidgeCV
# from sklearn.svm import LinearSVR

In [ ]:
# estimators = [('dtr',DecisionTreeRegressor(random_state=0)),
#               ('xgb',XGBRegressor(learning_rate=0.1,max_depth=4)),
#               ('lr', RidgeCV()),
#               ('rfr',RandomForestRegressor(n_estimators=10,random_state=42))]
# reg = StackingRegressor(
#      estimators=estimators,
#      final_estimator=XGBRegressor(learning_rate=0.1,max_depth=4),
#      passthrough=True)


In [ ]:
# reg.fit(train_x,train_y)
# print(mean_squared_error(reg.predict(train_x),train_y,squared=False))
# print(mean_squared_error(reg.predict(val_x),val_y,squared=False))

In [ ]:
rf = RandomForestRegressor(n_estimators=50, max_depth=6, random_state=0)
clf = BaggingRegressor(rf, n_estimators=100, max_samples=0.1, random_state=25)
clf.fit(train_x, train_y)
print(mean_squared_error(clf.predict(train_x),train_y,squared=False))
print(mean_squared_error(clf.predict(val_x),val_y,squared=False))

0.46381473215043784
0.4656684529895096


In [ ]:
xgbr = XGBRegressor(learning_rate=0.1,max_depth=4,n_estimators=100)
xgbr.fit(train_x,train_y)
print(mean_squared_error(xgbr.predict(train_x),train_y,squared=False))
print(mean_squared_error(xgbr.predict(val_x),val_y,squared=False))

[04:59:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.4457197813619883
0.45503575252810136


In [ ]:
gbreg = GradientBoostingRegressor(random_state=0,)

gbreg.fit(train_x,train_y)
print(mean_squared_error(gbreg.predict(train_x),train_y,squared=False))
print(mean_squared_error(gbreg.predict(val_x),val_y,squared=False))

0.455522213373643
0.4588111207466268


In [ ]:
dtreg = DecisionTreeRegressor(random_state=0,)
dtreg.fit(train_x,train_y)
print(mean_squared_error(dtreg.predict(train_x),train_y,squared=False))
print(mean_squared_error(dtreg.predict(val_x),val_y,squared=False))

0.018214623581661066
0.6318580320780125


In [ ]:
from keras import models, layers

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
model = models.Sequential()
scaler.fit(train_x)
model.add(layers.Dense(8, activation='relu', input_shape=[train_x.shape[1]]))
model.add(layers.Dense(16, activation='relu'))

# output layer
model.add(layers.Dense(1))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
history = model.fit(scaler.transform(train_x),y_train, validation_split=0.2, epochs=20)

Epoch 1/20
1482/1482 [==============================] - 4s 3ms/step - loss: 0.5944 - mae: 0.5769 - val_loss: 0.3060 - val_mae: 0.4469
Epoch 2/20
1482/1482 [==============================] - 4s 3ms/step - loss: 0.2845 - mae: 0.4410 - val_loss: 0.2967 - val_mae: 0.4425
Epoch 3/20
1482/1482 [==============================] - 4s 3ms/step - loss: 0.2823 - mae: 0.4401 - val_loss: 0.2963 - val_mae: 0.4424
Epoch 4/20
1482/1482 [==============================] - 4s 3ms/step - loss: 0.2806 - mae: 0.4389 - val_loss: 0.3183 - val_mae: 0.4532
Epoch 5/20
1482/1482 [==============================] - 4s 3ms/step - loss: 0.2805 - mae: 0.4389 - val_loss: 0.3070 - val_mae: 0.4485
Epoch 6/20
1482/1482 [==============================] - 4s 3ms/step - loss: 0.2799 - mae: 0.4383 - val_loss: 0.2917 - val_mae: 0.4374
Epoch 7/20
1482/1482 [==============================] - 5s 3ms/step - loss: 0.2794 - mae: 0.4381 - val_loss: 0.2936 - val_mae: 0.4403
Epoch 8/20
1482/1482 [==============================] - 5s 3ms

In [ ]:
print(mean_squared_error(model.predict(scaler.transform(train_x)),train_y,squared=False))
print(mean_squared_error(model.predict(scaler.transform(val_x)),val_y,squared=False))

0.5452072963078822
0.5438258704210804


In [ ]:
rf = RandomForestRegressor(n_estimators=100,n_jobs=-1,verbose=1)
rf.fit(train_x,train_y)
print(mean_squared_error(rf.predict(train_x),train_y,squared=False))
print(mean_squared_error(rf.predict(val_x),val_y,squared=False))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  9.2min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


0.17105010921984948


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.3s finished


0.45346346014522154


### Best Model

In [ ]:
pred_y = xgbr.predict(all_to_test)
print(pred_y)

[2.0553598 2.0104961 2.2209315 ... 2.0284925 2.4464445 2.6174424]


In [ ]:
pred_y[pred_y<1] = 1
pred_y[pred_y>3] = 3

In [ ]:
pd.DataFrame({"id": id_test, "relevance": pred_y}).to_csv('submission.csv',index=False)


In [ ]:
from google.colab import files
files.download('submission.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>